We can establish our framework by defining the variables to be used.  We start with the outcome variable, $Y$.  Next, we consider out treatment variable, $A_k$, which takes on value 1 if there is treatment, and is 0 otherwise.  To begin, we will use treatment strategies that are exclusively treatment or exclusively no treatment, corresponding to $\overline{a} = (1,1,\dots,1) = \overline{1}$ and $\overline{a} = (0,0,\dots,0) = \overline{0}$ respectively.  The next measurable variable is $L$, which represents the covariate(s) to be included.  Note that both the covariates, $L$, and the outcome, $Y$ are affected by an unmeasured common cause, $U$.  The diagram below illustrates these relationships.  

![title](image4.png)

The purpose of this investigation is to measure the average causal effect of treatment, which can be estimated using 
$$ \mathbb{E}\big[Y^{a=1}\big] - \mathbb{E}\big[Y^{a=0}\big]$$ 

We want to build out the g-formula as follows  
$$ \mathbb{E} \big[Y_{i}^{\overline{a}}\big]  = \sum_{i} \mathbb{E} \big[Y_i \mid A_{i-1} = a_{i-1},  \; A_{i-2} = a_{i-2}, \; L_{i-1}, \; L_{i-2} \big]$$ 

We can do this by building out two models, one for $Y$ And one for $L$.  We will begin by using a continous $Y$ and a binary $L$ for simplicity.  

For $Y$, we will use a linear regression for a time delay of t=2.  The model will look something like this 

$$\mathbb{E} \big[Y_{t+1} \mid \overline{A}_t, \overline{L}_t, \overline{Y}_t \big] = \theta_{0,t} + \theta_1 Y_{t} + \theta_2 Y_{t-1} + \theta_3 A_{t}+ \theta_4 A_{t-1} + \theta_5 L_t + \theta_6 L_{t-1} $$ 

For each $L$, we will use a logistic regression, also calculated on a time delay of t=2.  This will give us something similar to 
$$ logistP\big[L_{t+1} \mid \overline{A}_t, \overline{L}_t, \overline{Y}_{t+1} \big] = \alpha_{0,t} + \alpha_1 Y_{t+1} + \alpha_2 Y_{t} + \alpha_3 Y_{t-1} +  \alpha_4 A_{t}+ \alpha_5 A_{t-1} + \alpha_6 L_t + \alpha_7 L_{t-1} $$ 


<!---what quadratic time term needed to be added here? --->

We first need to simulate the data. We will build out the covariates of the population.  For simplicity, we will use one binary covariate, $L_1$.  

The data is being simulated using the following equations, where U is an underlying confounder.  

$$logitP[L_k] = \alpha_0 + \alpha_1 \cdot L_{k-1} + \alpha_2 \cdot L_{k-2} + \alpha_3 A_{k-1} + \alpha_4 A_{K-2} + \alpha_5 U$$ 


$$ logit[A_k] = \beta_0 + \beta_1 L_{k} + \beta_2 L_{k-1} + \beta_3 A_{k-1} + \beta_4 A_{K-2} $$ 

Then, an end $Y$ value will be pulled from the following 
$$ Y \sim N(\mu = U, \sigma = 1) $$ 

In [207]:
import pandas as pd
import numpy as np
import scipy as sp
import sklearn as sk
import math
import csv
import statsmodels.api as sm
import statsmodels.formula.api as smf
import random

In [158]:
#########################################################################
##[FUNCTION] data_creation simulates data for a given number of 
## individuals(indiv) over a set amount of time (max_time), and can 
## include as many covariates as desired (number_of_covariates)

## -- need to create the functionality for multiple covariates

#########################################################################


def data_creation(indiv, max_time, number_of_covariates): 

    columns = ["indiv", "time","U", "A", "Y",  "L1"]
    df = pd.DataFrame(columns = columns)
    coefficient_df = pd.DataFrame(columns = ["indiv", "alpha_0", "alpha_1", \
                     "alpha_2", "alpha_3", "alpha_4", "alpha_5", "beta_0", \
                     "beta_1", "beta_2", "beta_3", "beta_4"])

    for ii in range(1,indiv+1):

        alpha_0 = np.random.uniform(low = -1, high = 1)
        alpha_1 = np.random.uniform(low = -1, high = 1)
        alpha_2 = np.random.uniform(low = -1, high = 1)
        alpha_3 = np.random.uniform(low = -1, high = 1)
        alpha_4 = np.random.uniform(low = -1, high = 1)
        alpha_5 = np.random.uniform(low = -1, high = 1)

        beta_0 = np.random.uniform(low = -1, high = 1)
        beta_1 = np.random.uniform(low = -1, high = 1)
        beta_2 = np.random.uniform(low = -1, high = 1)
        beta_3 = np.random.uniform(low = -1, high = 1)
        beta_4 = np.random.uniform(low = -1, high = 1)

        coefficient_df.loc[len(coefficient_df)+1] = [ii, alpha_0, alpha_1, alpha_2, \
                alpha_3, alpha_4, alpha_5, beta_0, beta_1, beta_2, beta_3, beta_4, ]

        for jj in range(0, time+1): 

            ## creating an unobserved variable that affects covariates 
            U = np.random.uniform(low = 0.1, high = 1)

            if jj == 0: 
                x_L = alpha_0 + alpha_5*U 
                L1 = np.random.binomial(n=1, p = np.exp(x_L)/(1+np.exp(x_L)))

                x_A = beta_0 + beta_1*L1 
                A = np.random.binomial(n=1, p = np.exp(x_A)/(1+np.exp(x_A)))

                df.loc[len(df)+1] = [ii, jj, U, A, "NaN",  L1]

            elif jj == 1: 
                x_L = alpha_0 + alpha_1*float(df["L1"][(df.time == jj-1) & (df.indiv == ii)]) \
                    +alpha_3*float(df["A"][(df.time == jj-1) & (df.indiv == ii)])+ alpha_5*U 
                
                L1 = np.random.binomial(n=1, p = np.exp(x_L)/(1+np.exp(x_L)))

                
                x_A = beta_0 + beta_1*L1 + beta_2*float(df["L1"][(df.time == jj-1) & (df.indiv == ii)])\
                    + beta_3*float(df["A"][(df.time == jj-1) & (df.indiv == ii)])
                
                A = np.random.binomial(n=1, p = np.exp(x_A)/(1+np.exp(x_A)))

                df.loc[len(df)+1] = [ii, jj, U, A, "NaN", L1]

            else: 
                x_L = alpha_0 + alpha_1*float(df["L1"][(df.time == jj-1) & (df.indiv == ii)])\
                    +alpha_2*float(df["L1"][(df.time == jj-2) & (df.indiv == ii)])\
                    +alpha_3*float(df["A"][(df.time == jj-1) & (df.indiv == ii)])\
                    +alpha_4*float(df["A"][(df.time == jj-2) & (df.indiv == ii)])+ alpha_5*U 
                
                L1 = np.random.binomial(n=1, p = np.exp(x_L)/(1+np.exp(x_L)))


                x_A = beta_0 + beta_1*L1 + beta_2*float(df["L1"][(df.time == jj-1) & (df.indiv == ii)])\
                    + beta_3*float(df["A"][(df.time == jj-1) & (df.indiv == ii)])+ \
                    beta_4*float(df["A"][(df.time == jj-2) & (df.indiv == ii)])
                
                A = np.random.binomial(n=1, p = np.exp(x_A)/(1+np.exp(x_A)))

                if jj == time: 
                    Y = np.random.normal(loc = U, scale = 1)
                    df.loc[len(df)+1] = [ii, jj, U, A, Y, L1]

                else: 
                    df.loc[len(df)+1] = [ii, jj, U, A, "NaN", L1]

    # creating shifted values 
    for kk in range(1,time+1):
        df["L1_"+str(kk)] = df.L1.shift(kk)
        df["A_"+str(kk)] = df.A.shift(kk)

    return(df, coefficient_df); 

In [159]:
## establishing constants 
indiv = 500   ## number of individuals in study 
max_time = 10 ## number of time points being considered 
t_delay = 2 ## number of time delays included in model 

## Building out simulated data 
[df, coefficient_df] = data_creation(indiv,max_time, 2)
df.head(25) 

,indiv,time,U,A,Y,L1,L1_1,A_1,L1_2,A_2,...,L1_6,A_6,L1_7,A_7,L1_8,A_8,L1_9,A_9,L1_10,A_10
1,1.0,0.0,0.494213,1.0,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,1.0,0.519367,0.0,NaN,1.0,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,2.0,0.423268,0.0,NaN,0.0,1.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,3.0,0.567757,0.0,NaN,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,4.0,0.738277,1.0,NaN,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1.0,5.0,0.646786,1.0,NaN,1.0,1.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1.0,6.0,0.213084,0.0,NaN,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1.0,7.0,0.214904,0.0,NaN,0.0,1.0,0.0,1.0,1.0,...,1.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
9,1.0,8.0,0.524368,0.0,NaN,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN
10,1.0,9.0,0.570790,0.0,NaN,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,NaN,NaN


Starting with the model for Y, we use a linear regression to get the following model, 
$$\mathbb{E} \big[Y \mid \overline{A}_t, \overline{L}_t \big] = \theta_{0,t} + \theta_1 A_{t}+ \theta_2 A_{t-1} + \theta_3 L_t + \theta_4 L_{t-1} $$ 

In [214]:
temp_df = df[df.time == max_time]
train_columns ='+'.join(map(str, np.append(list(df)[3],list(df)[5:])))
temp_df = temp_df.astype(float)
Y_model = smf.ols("Y~"+train_columns, data=temp_df).fit()
Y_model.params

Intercept    0.599424
A           -0.003975
L1          -0.051260
L1_1        -0.026176
A_1         -0.113576
L1_2        -0.238071
A_2          0.062454
L1_3        -0.144556
A_3         -0.138073
L1_4         0.030054
A_4         -0.058287
L1_5         0.044423
A_5          0.028993
L1_6         0.051803
A_6         -0.086675
L1_7         0.035478
A_7         -0.034772
L1_8         0.054085
A_8          0.364999
L1_9         0.114728
A_9         -0.006222
L1_10       -0.026078
A_10         0.059047
dtype: float64

In [141]:
## Creating model for L1
columns = ["time", "gamma_0", "gamma_1", "gamma_2", "gamma_3", "gamma_4"]
train_columns = ["L1_1", "L1_2", "A_1", "A_2"]
L1_model_df = pd.DataFrame(columns = columns)

for ii in range(1, time+1): 
    temp_df = df[df.time == ii]   
    if ii == 1: 
        L1_model = sm.Logit(np.asarray(temp_df["L1"]), np.asarray(sm.add_constant(temp_df[["L1_1", "A_1"]]))).fit()
        L1_model_df = L1_model_df.append(pd.DataFrame([ii] + [L1_model.params[i] for i in range(0,2)] + ["Nan"] + [L1_model.params[2]] + ["Nan"], index = columns).transpose(), ignore_index=True)
    else: 
        L1_model = sm.Logit(np.asarray(temp_df["L1"]), np.asarray(sm.add_constant(temp_df[train_columns]))).fit()
        L1_model_df = L1_model_df.append(pd.DataFrame([ii] + [L1_model.params[i] for i in range(0,5)], index = columns).transpose(), ignore_index=True)
    

Optimization terminated successfully.
         Current function value: 0.690366
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.685133
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.669788
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.665168
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.670384
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.669131
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.671001
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.671317
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.676920
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.669211
  

Optimization terminated successfully.
         Current function value: 0.678727
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.666155
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.664800
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.652955
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.676538
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.682597
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.668675
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.678641
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.669143
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.676177
  

Now, using these three models, we can calculate the following expectations

$$ \mathbb{E} \big[Y_{i}^{\overline{a}}\big]  = \sum_{i} \mathbb{E} \big[Y_i \mid A_{i-1} = a_{i-1},  \; A_{i-2} = a_{i-2}, \; L_{1, i-1}, \; L_{1, i-2}, \; L_{2, i-1}, \; L_{2, i-2} \big]$$ 

for the respective $\bar{a} = \bar{0}$ and $\bar{a} = \bar{1}$

In [340]:
list1 = [A_test[time-i] for i in range(1,time+2)]
list2 = [values[time-i] for i in range(1,time+2)]

result = [None]*(len(list1)+len(list2))
result[::2] = list1
result[1::2] = list2
result = [1] + result
result

[1,
 1,
 -2.3913544552245307,
 1,
 -2.1231705656355162,
 1,
 -2.3874233388513826,
 1,
 -1.8591041498871959,
 1,
 -1.4920982159309506,
 1,
 -1.284973941776768,
 1,
 -0.62473497808995726,
 1,
 -0.33400168459802226,
 1,
 -0.14440769693841435,
 1,
 0.0,
 1,
 -2.3913544552245307]

In [344]:
reps = 10000
final_results_1 = np.empty(reps) 

### establishing treatment of interest
A_test = [1]*time

for ii in range(0,reps):
    values = np.empty(time)
    values[0] = random.choice(list(df["L1"][df["time"] == 0]))
    if values[0] == 0: 
        prod = 1-np.mean(list(df["L1"][df["time"] == 0]))
    else: 
        prod = np.mean(list(df["L1"][df["time"] == 0]))
    
    for jj in range(1, time):
        if jj == 1: 
            values[1] = np.sum(np.array([L1_model_df.ix[jj-1,][i] for i in [1,2,4]])*[1,values[jj-1],A_test[jj-1]])
        else: 
            values[jj] = np.sum(np.array([L1_model_df.ix[jj-1,][i] for i in range(1,6)])*[1,values[jj-1],values[jj-2], A_test[jj-1], A_test[jj-2]])
        prod = prod*(np.exp(values[jj])/(1+np.exp(values[jj])))
        
    list1 = [A_test[time-i] for i in range(1,time+2)]
    list2 = [values[time-i] for i in range(1,time+2)]

    result = [None]*(len(list1)+len(list2))
    result[::2] = list1
    result[1::2] = list2
    result = [1] + result
    
    Y_exp = np.sum(np.array(Y_model.params)*result)
    
    final_results_1[ii] = prod*Y_exp

mean_1 = np.mean(final_results_1)



final_results_0 = np.empty(reps) 
A_test = [0]*time

for ii in range(0,reps):
    values = np.empty(time)
    values[0] = random.choice(list(df["L1"][df["time"] == 0]))
    if values[0] == 0: 
        prod = 1-np.mean(list(df["L1"][df["time"] == 0]))
    else: 
        prod = np.mean(list(df["L1"][df["time"] == 0]))
    
    for jj in range(1, time):
        if jj == 1: 
            values[1] = np.sum(np.array([L1_model_df.ix[jj-1,][i] for i in [1,2,4]])*[1,values[jj-1],A_test[jj-1]])
        else: 
            values[jj] = np.sum(np.array([L1_model_df.ix[jj-1,][i] for i in range(1,6)])*[1,values[jj-1],values[jj-2], A_test[jj-1], A_test[jj-2]])
        prod = prod*(np.exp(values[jj])/(1+np.exp(values[jj])))
        
    list1 = [A_test[time-i] for i in range(1,time+2)]
    list2 = [values[time-i] for i in range(1,time+2)]

    result = [None]*(len(list1)+len(list2))
    result[::2] = list1
    result[1::2] = list2
    result = [1] + result
    
    Y_exp = np.sum(np.array(Y_model.params)*result)
    
    final_results_0[ii] = prod*Y_exp

mean_0 = np.mean(final_results_0)
final_answer = mean_1 - mean_0 
final_answer

-5.2346948761580905e-07

Calculating the final difference for each time, using the following
$$ \mathbb{E}\big[Y^{a=1}\big] - \mathbb{E}\big[Y^{a=0}\big]$$ 

In [320]:
A_test = [1]*time
values = range(0,time)

list1 = [A_test[time-i] for i in range(1,time+1)]
list2 = [values[time-i] for i in range(1,time+1)]

result = [None]*(len(list1)+len(list2))
result[::2] = list1
result[1::2] = list2
result
result

[1, 9, 1, 8, 1, 7, 1, 6, 1, 5, 1, 4, 1, 3, 1, 2, 1, 1, 1, 0]

In [59]:
type(list(L_0s))

list

In [ ]:
## DONT THINK I NEED THIS 
## Creating model for A 
columns = ["time", "delta_0", "delta_1", "delta_2", "delta_3", "delta_4"] 
train_columns = ["L1", "L1_1", "A_1", "A_2"]
A_model_df = pd.DataFrame(columns = columns)

for ii in range(1, time+1): 
    temp_df = df[df.time == ii]
    if ii == 1: 
        A_model = sm.Logit(np.asarray(temp_df["A"]), np.asarray(sm.add_constant(temp_df[["L1", "L1_1", "A_1"]]))).fit()
        A_model_df = A_model_df.append(pd.DataFrame([ii] + [A_model.params[i] for i in range(0,4)] + ["Nan"], index = columns).transpose(), ignore_index=True)
    else:
        A_model = sm.Logit(np.asarray(temp_df["A"]), np.asarray(sm.add_constant(temp_df[train_columns]))).fit()
        A_model_df = A_model_df.append(pd.DataFrame([ii] + [A_model.params[i] for i in range(0,5)], index = columns).transpose(), ignore_index=True)
   